In [56]:
## IMPORT NECESSARY PYTHON MODULES:
from sqlalchemy import create_engine
import psycopg2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os


In [57]:
## CONNECT TO SQL DATABASE:
# Provide database info
dbname = os.environ['dbname']
pswd = os.environ['pswd']
username = os.environ['username']
# construct a connection to a database
engine = create_engine('postgresql://%s:%s@localhost/%s'%(username,pswd,dbname))
# connect to the database
con = None
con = psycopg2.connect(database = dbname, user = username, host='localhost', password=pswd)

In [58]:
from mapbox import Geocoder

MAPBOX_ACCESS_TOKEN=os.environ['MAPBOX_ACCESS_TOKEN']

geocoder = Geocoder(access_token=MAPBOX_ACCESS_TOKEN)

#response_start = geocoder.forward("360 Elizabeth St, San Francisco, CA 94114")
response_start = geocoder.forward("501 Twin Peaks Blvd, San Francisco, CA 94114")
start_coords = response_start.geojson()['features'][0]['geometry']['coordinates']
lat1 = start_coords[0]
long1 = start_coords[1]

response_end = geocoder.forward("1720 Market St, San Francisco, CA 94102")
end_coords = response_end.geojson()['features'][0]['geometry']['coordinates']
lat2 = end_coords[0]
long2 = end_coords[1]

In [63]:
cur_get_ids_from_ll_start = con.cursor()
cur_get_ids_from_ll_end = con.cursor()
cur_get_ids_from_ll_start.execute("""select id from ways_vertices_pgr order by st_distance(the_geom, st_setsrid(st_makepoint(%s, %s), 4326)) limit 1;""", (lat1, long1))
cur_get_ids_from_ll_end.execute("""select id from ways_vertices_pgr order by st_distance(the_geom, st_setsrid(st_makepoint(%s, %s), 4326)) limit 1;""", (lat2, long2))

id_start = cur_get_ids_from_ll_start.fetchall()
id_end = cur_get_ids_from_ll_end.fetchall()

[(126971L,)]

## Cell below needs editing to return cal_cost and length_ft columns. The totals from these columns then need to be calculated and displayed in the web app

In [61]:
curX = con.cursor()
curX.execute("""SELECT id, lon, lat, route.cost, route.edge, new_ways.length_ft
   FROM ways_vertices_pgr
   JOIN
   (SELECT * FROM pgr_dijkstra('
    SELECT gid AS id,
         source,
         target,
         length_ft AS cost,
         length_ft
        FROM ways_cals',
    %s, %s, directed := false )) AS route
   ON
   ways_vertices_pgr.id = route.node
   JOIN (SELECT gid, length_ft FROM ways_cals) AS new_ways
   ON new_ways.gid = route.edge
   ORDER BY seq;""", (id_start, id_end))
Path = curX.fetchall()
Path[:10]

[(35933L,
  Decimal('-122.44559300'),
  Decimal('37.75555970'),
  828.402540289306,
  40893L,
  828.402540289306),
 (46680L,
  Decimal('-122.44694230'),
  Decimal('37.75649950'),
  484.288764908419,
  40892L,
  484.288764908419),
 (20406L,
  Decimal('-122.44790270'),
  Decimal('37.75755680'),
  127.870360206564,
  122107L,
  127.870360206564),
 (46866L,
  Decimal('-122.44817210'),
  Decimal('37.75783530'),
  459.076267601774,
  119757L,
  459.076267601774),
 (8353L,
  Decimal('-122.44848470'),
  Decimal('37.75904990'),
  159.934581691944,
  40891L,
  159.934581691944),
 (43513L,
  Decimal('-122.44794010'),
  Decimal('37.75912460'),
  166.960532894734,
  40890L,
  166.960532894734),
 (11277L,
  Decimal('-122.44744390'),
  Decimal('37.75935150'),
  334.474006353616,
  40889L,
  334.474006353616),
 (4451L,
  Decimal('-122.44695300'),
  Decimal('37.76017000'),
  330.15192105246,
  40888L,
  330.15192105246),
 (22314L,
  Decimal('-122.44651720'),
  Decimal('37.76100470'),
  182.133848046542

## The following cell, given the cell above (which has been edited to return the cost column in the Path variable), gives the total of the "cal_cost" variable along the path. NOTE: this does NOT equal the calories burned!

In [55]:
Total_Cal = 0
Total_Dist = 0
for i in range(len(Path)):
    Total_Cal += Path[i][3]
    Total_Dist += Path[i][5]
Total_Cal  


11700.100467986296

In [33]:
Route_Coords = []
for i in range(len(Path)):
    Route_Coords.append([float(Path[i][1]),float(Path[i][2])])

len(Route_Coords)

65

In [34]:
for i in range(len(Route_Coords)):
    Route_Coords[i][0], Route_Coords[i][1] = Route_Coords[i][1], Route_Coords[i][0]
    
#Route_Coords

In [35]:
Route_str = "["
for i in range(len(Route_Coords)):
    Route_str += "["
    Route_str += str(Route_Coords[i][0])
    Route_str += ","
    Route_str += str(Route_Coords[i][1])
    Route_str += "]"
    Route_str += ","
Route_str = Route_str[:-1]
Route_str += "];"
    

In [36]:
Route_str

'[[37.7555597,-122.445593],[37.7564995,-122.4469423],[37.7575568,-122.4479027],[37.7578353,-122.4481721],[37.7590499,-122.4484847],[37.7591246,-122.4479401],[37.7593515,-122.4474439],[37.76017,-122.446953],[37.7610047,-122.4465172],[37.7614825,-122.4466073],[37.7615697,-122.4466512],[37.7617944,-122.4467789],[37.7618173,-122.4463856],[37.7618593,-122.4456723],[37.7618829,-122.4453093],[37.7619538,-122.4442481],[37.7620016,-122.4433594],[37.762029,-122.4429086],[37.7621959,-122.4401689],[37.7622371,-122.439492],[37.7623024,-122.4384372],[37.7623466,-122.437694],[37.7623662,-122.437372],[37.7626492,-122.435407],[37.76271,-122.4352027],[37.7628033,-122.4350308],[37.7629997,-122.4347302],[37.7640404,-122.4334212],[37.764226,-122.4331992],[37.7643025,-122.4331078],[37.764477,-122.4329031],[37.7657839,-122.4312174],[37.7659327,-122.4310488],[37.7660108,-122.4309628],[37.7664924,-122.4303682],[37.7673194,-122.4293052],[37.7674793,-122.4291018],[37.7675698,-122.4289903],[37.7675838,-122.428973

In [41]:
User_Total_Dist_mi = Total_Dist/5280
User_Total_Dist_mi = round(User_Total_Dist_mi,3)
User_Total_Dist_mi


2.216

In [51]:
Total_Dist = round(Total_Dist,3)

In [52]:
Total_Dist

15623.0